# 0. Import

In [1]:
import os
import re
import shutil
import sqlite3
import pandas as pd

# 1. Copy CSVs to New Dataset Folder

In [2]:
# Make datasets folder

os.chdir("../")

dataset_dir = 'datasets'

try:
    mkdir = 'mkdir {0}'.format(dataset_dir)
    os.system(mkdir)
    print("Folder created successfully")
except:
    print("Directory already exists or there is another error")

Folder created successfully


In [3]:
# Change directory

os.chdir("csv")
print("Directory changed to csv folder")

Directory changed to csv folder


In [4]:
# Copy CSVs into datasets folder

src_dir = "."
dst_dir = "../datasets"

try:
    for root, dirs, files in os.walk(src_dir):
        for f in files:
            if f.endswith('.csv'):
                shutil.copy(os.path.join(root,f), dst_dir)
            print(f"File '{f}' copied successfully")
except:
    print("There are no CSV files to copy")

File 'Company Client List.csv' copied successfully
File 'Company Orders.csv' copied successfully


# 2. Read Each CSV into a DataFrame

In [5]:
# Make list of CSVs

csv_files = []

for file in os.listdir(os.getcwd()):
    if file.endswith('.csv'):
        csv_files.append(file)

In [6]:
# Read CSVs 

data_path = '../'+dataset_dir+'/'

df = {}
for file in csv_files:
    try:
        df[file] = pd.read_csv(data_path+file)
        print(f"{file} read successfully")
    except UnicodeDecodeError:
        df[file] = pd.read_csv(data_path+file, encoding="ISO-8859-1")

Company Client List.csv read successfully
Company Orders.csv read successfully


In [7]:
csv_files

['Company Client List.csv', 'Company Orders.csv']

In [8]:
# Check dfs

df['Company Client List.csv'].head()

,client number,Company name,Contact first name,contact last name,Email,Phone
0,100,Ainyx,Timmie,Axston,taxston1y@biblegateway.com,167-281-7420
1,101,Aivee,Melli,Bartolomeotti,mbartolomeotti25@webs.com,507-785-4101
2,102,Avamba,Deb,Petruszka,dpetruszkas@1und1.de,496-125-6029
3,103,Avaveo,Dick,Borgnol,dborgnol1m@sbwire.com,229-807-3736
4,104,Babbleopia,Bendick,Sawkin,bsawkinr@cbsnews.com,447-459-8541


In [9]:
df['Company Orders.csv'].head()

,Company name,Order Number,Order amount,order date,paid?
0,Zoomcast,485495,$2096568.72,2/21/2020,False
1,Buzzbean,175468,$1607033.03,6/10/2021,True
2,Twitternation,6841,$2938905.06,6/2/2021,False
3,Topiczoom,335932,$2687554.82,8/17/2021,True
4,Youbridge,81726,$2206870.49,12/17/2020,False


# 3. Send Clean Version of DataFrame to Database

In [10]:
# Change directory

os.chdir("..")

os.listdir(os.getcwd())

['csv',
 'database',
 'datasets',
 'notebooks',
 'original_csvs',
 'python',
 'tableau',
 'test']

In [11]:
# Make database folder

database_dir = 'database'

try:
    mkdir = 'mkdir {0}'.format(database_dir)
    os.system(mkdir)
    print("Folder created successfully")
except:
    print("Directory already exists or there is another error")

Folder created successfully


In [12]:
os.listdir(os.getcwd())

['csv',
 'database',
 'datasets',
 'notebooks',
 'original_csvs',
 'python',
 'tableau',
 'test']

In [13]:
os.chdir(database_dir)

for k in csv_files:    
    dataframe = df[k]
    
    # Clean table name
    clean_table_name = k.lower().replace(" ", "_").replace(r"/""\\", "_")
    clean_table_name = re.sub(r'[^a-zA-Z0-9_.]','', clean_table_name)    
    
    #Clean table data
    dataframe.columns = [x.lower().replace(" ", "_").replace(r"/""\\", "_").strip() for x in dataframe.columns]
    dataframe.columns = dataframe.columns.str.replace('[^A-Za-z\s_.]+', '', regex=True)
    
    table_name = '{0}'.format(clean_table_name.split('.')[0])
    
    # Change data types
    new_dtypes = {
    'object': 'varchar',
    'float64': 'float',
    'int64': 'int',
    'datetime64': 'timestamp',
    'timedelta64[ns]': 'varchar'
    }
    
    col_strings = ", ".join("{} {}".format(n, d) for (n, d) in zip(dataframe.columns, dataframe.dtypes.replace(new_dtypes)))
    
    # Write to db with SQLite
    conn = sqlite3.connect('company.db')
    c = conn.cursor()

    c.execute('''
    CREATE TABLE IF NOT EXISTS table_name (
    col_strings    
    )
    ''')
    print('{0} was created successfully'.format(table_name))
    conn.commit()
    
    dataframe.to_sql(table_name, conn, if_exists='replace', index=False)
    
    conn.close()

company_client_list was created successfully
company_orders was created successfully


In [14]:
os.listdir(os.getcwd())

['.ipynb_checkpoints', 'company.db']